<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/testing/covid_gradient_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing per le Random Forest

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.ensemble import GradientBoostingRegressor

**Nomenclatura per i test**

In [21]:
path='/content/'
datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv','dataset_finale_medie_ponderate_rnd1.csv',
               'dataset_finale_medie_rnd220.csv','dataset_finale_medie_2_week_rnd220.csv','dataset_finale_medie_ponderate_rnd220.csv']

rdn_states= [1, 22, 777, 6654, 432145]
features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi',
                     'fut1_ricoverati_con_sintomi','fut2_ricoverati_con_sintomi',
                     'fut1_terapia_intensiva','fut2_terapia_intensiva',
                     'fut1_deceduti','fut2_deceduti',
                     'fut1_totale_ospedalizzati','fut2_totale_ospedalizzati',
                     'fut1_dimessi_guariti','fut2_dimessi_guariti']

coefficents=['R2','RMSE','MAXERR']

**Creazione indici per il dataset comune nel quale verranno messi i dati dei test**


In [22]:
col=[]
inde=[]
for feature in features_prediction:
  for coeff in coefficents:
    col.append(coeff+'_'+feature)
    #print(f"Columns_names\t{coeff+'_'+feature}")

for rnd in rdn_states:
  for path in datasets_names:
    inde.append(path+':'+str(rnd))
    #print(f"indexes_\t{path+':'+str(rnd)}")

df_results=pd.DataFrame(columns=col,index=inde)
print(df_results.columns)
print(df_results.index)

Index(['R2_fut1_nuovi_positivi', 'RMSE_fut1_nuovi_positivi',
       'MAXERR_fut1_nuovi_positivi', 'R2_fut2_nuovi_positivi',
       'RMSE_fut2_nuovi_positivi', 'MAXERR_fut2_nuovi_positivi',
       'R2_fut1_ricoverati_con_sintomi', 'RMSE_fut1_ricoverati_con_sintomi',
       'MAXERR_fut1_ricoverati_con_sintomi', 'R2_fut2_ricoverati_con_sintomi',
       'RMSE_fut2_ricoverati_con_sintomi',
       'MAXERR_fut2_ricoverati_con_sintomi', 'R2_fut1_terapia_intensiva',
       'RMSE_fut1_terapia_intensiva', 'MAXERR_fut1_terapia_intensiva',
       'R2_fut2_terapia_intensiva', 'RMSE_fut2_terapia_intensiva',
       'MAXERR_fut2_terapia_intensiva', 'R2_fut1_deceduti',
       'RMSE_fut1_deceduti', 'MAXERR_fut1_deceduti', 'R2_fut2_deceduti',
       'RMSE_fut2_deceduti', 'MAXERR_fut2_deceduti',
       'R2_fut1_totale_ospedalizzati', 'RMSE_fut1_totale_ospedalizzati',
       'MAXERR_fut1_totale_ospedalizzati', 'R2_fut2_totale_ospedalizzati',
       'RMSE_fut2_totale_ospedalizzati', 'MAXERR_fut2_totale_osped

In [23]:
# datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv']
# rdn_states= [1,22]
# features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi']


In [24]:
for rnd in rdn_states:
  for dataset in datasets_names:
    for feature in features_prediction:

      df_2 = pd.read_csv(dataset, parse_dates=["Data"], infer_datetime_format=True)
      rnd_state = rnd
      predictor_columns = [
            'pass_Ammoniaca', 
            'pass_Benzene',
            'pass_Biossido di Azoto', 
            'pass_Biossido di Zolfo',
            'pass_Monossido di Azoto', 
            'pass_Monossido di Carbonio',
            'pass_Ossidi di Azoto', 
            'pass_Ozono', 
            'pass_PM10 (SM2005)',
            'pass_Particelle sospese PM2.5',
            'pass_Radiazione Globale', 
            'pass_Temperatura',
            'pass_deceduti',
            'pass_nuovi_positivi',
            'pass_ricoverati_con_sintomi', 
            'pass_tamponi']
      ################################################################################
      train_2 = df_2.sample(frac=0.70, random_state=rnd_state)
      test_2 = df_2.drop(train_2.index)

      train_X_2 = train_2[predictor_columns]
      train_y_2 = train_2[feature]

      test_X_2 = test_2[predictor_columns]
      test_y_2 = test_2[feature]
      ################################################################################
      gb_regressor = GradientBoostingRegressor(random_state=rnd_state)
      ################################################################################
      
      ################################################################################
      pipeline = Pipeline([("imp", SimpleImputer(missing_values=np.nan, strategy="mean")), 
                     ("gbr", gb_regressor)])
      
      loss_list = ["ls", "lad"]
      n_estimators_list = [100]
      criterion_list = ["mse", "friedman_mse"]
      min_samples_split_list = [2, 3]
      min_samples_leaf_list = [1, 2]
      max_depth_list = [3, 5]
      max_features_list = ["sqrt", "auto"]

      param_grid = {"gbr__loss": loss_list,
              "gbr__n_estimators": n_estimators_list,
              "gbr__criterion": criterion_list,
              "gbr__min_samples_split": min_samples_split_list,
              "gbr__min_samples_leaf": min_samples_leaf_list,
              "gbr__max_depth": max_depth_list,
              "gbr__max_features": max_features_list}
      ################################################################################
      kfold_splits = 4
      grid_regressor_2 = GridSearchCV(estimator=pipeline,  
                              n_jobs=-1, 
                              verbose=1,
                              return_train_score=True,
                            cv=KFold(n_splits=kfold_splits, shuffle=True, random_state=rnd_state),
                              param_grid=param_grid)
      ################################################################################
      grid_regressor_2.fit(imp.transform(train_X_2), train_y_2)
      ################################################################################
      best_regressor_2 = grid_regressor_2.best_estimator_

      imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
      imputer = imputer.fit(test_X_2)
      test_X_imp = imputer.transform(test_X_2)

      predicted_2 = best_regressor_2.predict(test_X_imp)
      ################################################################################
      r2=float("{:.3f}".format(r2_score(test_y_2, predicted_2)))
      rmse=float("{:.3f}".format(mean_squared_error(test_y_2, predicted_2, squared=False)))
      maxerr=float("{:.3f}".format(max_error(test_y_2, predicted_2)))
      print(f"R2: \t\t{r2}")
      print(f"RMSE: \t\t{rmse}")
      print(f"MAX ERR: \t{maxerr}")
      ################################################################################
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[0]+'_'+feature]] = r2
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[1]+'_'+feature]] = rmse
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[2]+'_'+feature]] = maxerr

Fitting 4 folds for each of 64 candidates, totalling 256 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed:   21.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2: 		0.888
RMSE: 		225.713
MAX ERR: 	874.253
Fitting 4 folds for each of 64 candidates, totalling 256 fits


[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed:   20.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2: 		0.772
RMSE: 		366.383
MAX ERR: 	1747.811
Fitting 4 folds for each of 64 candidates, totalling 256 fits


...

[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed:   18.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2: 		0.996
RMSE: 		1429.781
MAX ERR: 	4401.317
Fitting 4 folds for each of 64 candidates, totalling 256 fits


[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:   10.4s


R2: 		0.996
RMSE: 		1467.454
MAX ERR: 	3828.712


[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed:   18.3s finished


In [25]:
df_results.head(10)

,R2_fut1_nuovi_positivi,RMSE_fut1_nuovi_positivi,MAXERR_fut1_nuovi_positivi,R2_fut2_nuovi_positivi,RMSE_fut2_nuovi_positivi,MAXERR_fut2_nuovi_positivi,R2_fut1_ricoverati_con_sintomi,RMSE_fut1_ricoverati_con_sintomi,MAXERR_fut1_ricoverati_con_sintomi,R2_fut2_ricoverati_con_sintomi,RMSE_fut2_ricoverati_con_sintomi,MAXERR_fut2_ricoverati_con_sintomi,R2_fut1_terapia_intensiva,RMSE_fut1_terapia_intensiva,MAXERR_fut1_terapia_intensiva,R2_fut2_terapia_intensiva,RMSE_fut2_terapia_intensiva,MAXERR_fut2_terapia_intensiva,R2_fut1_deceduti,RMSE_fut1_deceduti,MAXERR_fut1_deceduti,R2_fut2_deceduti,RMSE_fut2_deceduti,MAXERR_fut2_deceduti,R2_fut1_totale_ospedalizzati,RMSE_fut1_totale_ospedalizzati,MAXERR_fut1_totale_ospedalizzati,R2_fut2_totale_ospedalizzati,RMSE_fut2_totale_ospedalizzati,MAXERR_fut2_totale_ospedalizzati,R2_fut1_dimessi_guariti,RMSE_fut1_dimessi_guariti,MAXERR_fut1_dimessi_guariti,R2_fut2_dimessi_guariti,RMSE_fut2_dimessi_guariti,MAXERR_fut2_dimessi_guariti
dataset_finale_medie_rnd1.csv:1,0.888,225.713,874.253,0.772,366.383,1747.81,0.978,596.244,2967.04,0.967,737.015,3558.85,0.983,61.544,282.505,0.984,61.458,191.111,0.993,442.487,1346.38,0.989,535.895,1344.39,0.98,621.825,2807.66,0.979,655.161,2434.64,0.998,1211.33,4298.57,0.998,1070.59,2402.51
dataset_finale_medie_2_week_rnd1.csv:1,0.863,250.281,754.657,0.802,342.027,1691.74,0.991,388.676,1225.47,0.985,497.954,1724.72,0.982,62.112,275.061,0.979,70.756,225.55,0.996,365.086,1134.12,0.995,387.27,1260.92,0.984,557.576,2059.43,0.979,654.633,2218.85,0.996,1502.6,4518.25,0.997,1404.18,3738.12
dataset_finale_medie_ponderate_rnd1.csv:1,0.865,247.751,808.128,0.781,358.964,1615.34,0.986,479.129,1746.2,0.976,633.4,2954.08,0.982,62.505,279.151,0.986,57.728,235.104,0.991,517.321,1406.43,0.991,505.442,1381.93,0.977,668.974,2900.03,0.978,677.429,3574.01,0.997,1361.35,3290.8,0.998,1095.39,3373.15
dataset_finale_medie_rnd220.csv:1,0.884,230.234,831.889,0.828,318.307,1414.17,0.98,563.055,1600.7,0.98,569.923,1529.13,0.984,59.973,179.377,0.976,75.419,284.076,0.985,656.03,2651.21,0.979,757.777,3263.76,0.986,527.787,1823.86,0.98,647.916,2424.27,0.997,1377.37,4056.88,0.997,1360.38,3154.64
dataset_finale_medie_2_week_rnd220.csv:1,0.863,250.281,754.657,0.802,342.027,1691.74,0.991,388.676,1225.47,0.985,497.954,1724.72,0.982,62.112,275.061,0.979,70.756,225.55,0.996,365.086,1134.12,0.995,387.27,1260.92,0.984,557.576,2059.43,0.979,654.633,2218.85,0.996,1502.6,4518.25,0.997,1404.18,3738.12
dataset_finale_medie_ponderate_rnd220.csv:1,0.85,261.416,811.576,0.854,293.134,1332.25,0.983,512.297,1643.77,0.979,591.414,2270.39,0.983,60.888,209.878,0.992,43.771,178.683,0.987,632.831,2366.97,0.984,663.559,2270.76,0.984,565.278,1959.43,0.98,647.006,2761.16,0.998,1254.7,2840.98,0.995,1840.12,4891.42
dataset_finale_medie_rnd1.csv:22,0.851,240.297,874.757,0.783,386.637,1756.79,0.987,476.35,1566.79,0.987,491.868,1537.31,0.99,47.833,201.632,0.989,50.624,182.334,0.993,407.083,1420.75,0.994,370.531,1323.38,0.99,476.472,1839.93,0.989,513.47,1660.32,0.999,773.912,1939.88,0.997,1478.76,7563.71
dataset_finale_medie_2_week_rnd1.csv:22,0.837,250.816,950.517,0.805,366.999,1654.74,0.975,669.204,2576.73,0.987,494.565,1768.34,0.984,60.271,190.415,0.995,33.098,104.917,0.996,310.593,1043.49,0.994,358.984,1044.06,0.985,570.613,1895.24,0.988,527.704,1793.05,0.998,1019.42,4625.12,0.996,1490.85,8227.51
dataset_finale_medie_ponderate_rnd1.csv:22,0.883,213.012,774.148,0.769,399.182,1815.59,0.989,452.19,1566.99,0.972,722.565,2837.23,0.988,53.654,218.722,0.984,60.609,228.164,0.996,306.249,1261.37,0.992,413.009,1488.11,0.992,418.552,1224.81,0.987,548.214,1976.2,0.998,1035.39,2299.05,0.997,1438.1,6787.91
dataset_finale_medie_rnd220.csv:22,0.907,189.943,633.363,0.788,382.124,1705.57,0.99,424.312,1239.88,0.987,497.407,1602.02,0.985,58.46,243.514,0.991,46.694,154.25,0.996,316.289,1377.63,0.994,369.22,1514.74,0.991,444.469,1427.88,0.984,600.594,2201.6,0.998,987.932,3665.12,0.996,1576.08,7677.91


In [27]:
df_results.to_csv('covid_gradient_test.csv')